*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Text Classification of MultiNLI Sentences using different Transformer models

In [ ]:
import sys
import os
import json
import pandas as pd
import numpy as np
import scrapbook as sb
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from tqdm import tqdm
from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.models.transformers.sequence_classification import (
    SequenceClassifier,
    Processor,
)
from utils_nlp.common.timer import Timer

## Introduction
In this notebook, we fine-tune and evaluate a number of pretrained models on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.

We use a [sequence classifier](../../utils_nlp/models/transformers/sequence_classification.py) that wraps [Hugging Face's PyTorch implementation](https://github.com/huggingface/transformers) of different transformers, like [BERT](https://github.com/google-research/bert), [XLNet](https://github.com/zihangdai/xlnet), and [RoBERTa](https://github.com/pytorch/fairseq).

In [32]:
# notebook parameters
DATA_FOLDER = "./temp"
CACHE_DIR = "./temp"
DEVICE = "cuda"
NUM_EPOCHS = 1
BATCH_SIZE = 16
NUM_GPUS = 2
MAX_LEN = 150
TRAIN_DATA_FRACTION = 0.15
TEST_DATA_FRACTION = 0.15
TRAIN_SIZE = 0.75
LABEL_COL = "genre"
TEXT_COL = "sentence1"

## Read Dataset
We start by loading a subset of the data. The following function also downloads and extracts the files, if they don't exist in the data folder.

The MultiNLI dataset is mainly used for natural language inference (NLI) tasks, where the inputs are sentence pairs and the labels are entailment indicators. The sentence pairs are also classified into *genres* that allow for more coverage and better evaluation of NLI models.

For our classification task, we use the first sentence only as the text input, and the corresponding genre as the label. We select the examples corresponding to one of the entailment labels (*neutral* in this case) to avoid duplicate rows, as the sentences are not unique, whereas the sentence pairs are.

In [4]:
df = load_pandas_df(DATA_FOLDER, "train")
df = df[df["gold_label"]=="neutral"]  # get unique sentences

In [5]:
df[[LABEL_COL, TEXT_COL]].head()

,genre,sentence1
0,government,Conceptually cream skimming has two basic dime...
4,telephone,yeah i tell you what though if you go price so...
6,travel,But a few Christian mosaics survive above the ...
12,slate,It's not that the questions they asked weren't...
13,travel,"Thebes held onto power until the 12th Dynasty,..."


We split the data for training and testing, sample a fraction for faster execution, and encode the class labels:

In [6]:
# split
df_train, df_test = train_test_split(df, train_size = TRAIN_SIZE, random_state=0)

/media/bleik2/miniconda3/envs/nlp_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
# sample
df_train = df_train.sample(frac=TRAIN_DATA_FRACTION).reset_index(drop=True)
df_test = df_test.sample(frac=TEST_DATA_FRACTION).reset_index(drop=True)

The examples in the dataset are grouped into 5 genres:

In [8]:
df_train[LABEL_COL].value_counts()

telephone     3146
fiction       2960
slate         2901
government    2893
travel        2826
Name: genre, dtype: int64

In [9]:
# encode labels
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(df_train[LABEL_COL])
labels_test = label_encoder.transform(df_test[LABEL_COL])

num_labels = len(np.unique(labels_train))

In [10]:
print("Number of unique labels: {}".format(num_labels))
print("Number of training examples: {}".format(df_train.shape[0]))
print("Number of testing examples: {}".format(df_test.shape[0]))

Number of unique labels: 5
Number of training examples: 14726
Number of testing examples: 4909


## Select Pretrained Models

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For text classification, the following pretrained models are supported.

In [11]:
pd.DataFrame({"model_name": SequenceClassifier.list_supported_models()})

,model_name
0,bert-base-uncased
1,bert-large-uncased
2,bert-base-cased
3,bert-large-cased
4,bert-base-multilingual-uncased
5,bert-base-multilingual-cased
6,bert-base-chinese
7,bert-base-german-cased
8,bert-large-uncased-whole-word-masking
9,bert-large-cased-whole-word-masking


## Fine-tune

Our wrappers make it easy to fine-tune different models in a unified way, hiding the preprocessing details that are needed before training. In this example, we're going to select the following models and use the same piece of code to fine-tune them on our genre classification task. Note that some models were pretrained on multilingual datasets and can be used with non-English datasets.

In [28]:
model_names = ["distilbert-base-uncased", "roberta-base", "xlnet-base-cased"]

For each pretrained model, we preprocess the data, fine-tune the classifier, score the test set, and store the evaluation results.

In [33]:
results = {}

for model_name in tqdm(model_names):
    
    # preprocess
    processor = Processor(model_name=model_name, cache_dir=CACHE_DIR)
    ds_train = processor.preprocess(
        df_train[TEXT_COL], labels_train, max_len=MAX_LEN
    )
    ds_test = processor.preprocess(df_test[TEXT_COL], None, max_len=MAX_LEN)

    # fine-tune
    classifier = SequenceClassifier(
        model_name=model_name, num_labels=num_labels, cache_dir=CACHE_DIR
    )
    with Timer() as t:
        classifier.fit(
            ds_train,
            device=DEVICE,
            num_epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            num_gpus=NUM_GPUS,
            verbose=False,
        )
    train_time = t.interval / 3600

    # predict
    preds = classifier.predict(
        ds_test, device="cuda", batch_size=BATCH_SIZE, num_gpus=NUM_GPUS
    )

    # eval
    accuracy = accuracy_score(labels_test, preds)
    class_report = classification_report(
        labels_test, preds, target_names=label_encoder.classes_, output_dict=True
    )

    # save results
    results[model_name] = {
        "accuracy": accuracy,
        "f1-score": class_report["macro avg"]["f1-score"],
        "time(hrs)": train_time,
    }


  0%|          | 0/3 [00:00<?, ?it/s]I1002 17:19:07.367456 140305852307264 tokenization_utils.py:373] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./temp/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I1002 17:19:16.283660 140305852307264 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at ./temp/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1002 17:19:16.285341 140305852307264 configuration_utils.py:168] Model config {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "n_heads": 12,
  "n_layers": 6,
  "num_lab

Loss:1.613472


                                     
  0%|          | 0/3 [00:28<?, ?it/s]

Loss:1.296651


                                     
  0%|          | 0/3 [00:33<?, ?it/s]

...

Loss:0.332851


                                     
 67%|██████▋   | 2/3 [36:31<08:58, 538.28s/it]

Loss:0.042949




Evaluating:   0%|          | 0/154 [00:00<?, ?it/s]

Evaluating:   1%|          | 1/154 [00:00<01:53,  1.35it/s]

Evaluating:   1%|▏         | 2/154 [00:01<01:43,  1.47it/s]

Evaluating:   2%|▏         | 3/154 [00:01<01:37,  1.56it/s]

Evaluating:   3%|▎         | 4/154 [00:02<01:31,  1.64it/s]

Evaluating:   3%|▎         | 5/154 [00:02<01:28,  1.69it/s]

Evaluating:   4%|▍         | 6/154 [00:03<01:25,  1.74it/s]

Evaluating:   5%|▍         | 7/154 [00:04<01:24,  1.74it/s]

Evaluating:   5%|▌         | 8/154 [00:04<01:22,  1.76it/s]

Evaluating:   6%|▌         | 9/154 [00:05<01:20,  1.79it/s]

Evaluating:   6%|▋         | 10/154 [00:05<01:19,  1.81it/s]

Evaluating:   7%|▋         | 11/154 [00:06<01:19,  1.79it/s]

Evaluating:   8%|▊         | 12/154 [00:06<01:27,  1.62it/s]

Evaluating:   8%|▊         | 13/154 [00:07<01:24,  1.67it/s]

Evaluating:   9%|▉         | 14/154 [00:08<01:23,  1.68it/s]

Evaluating:  10%|▉         | 15/154 [00:08<01:21,  1.71it/s]

Evaluating:  10%|█      

Evaluating:  78%|███████▊  | 120/154 [01:06<00:18,  1.84it/s]

Evaluating:  79%|███████▊  | 121/154 [01:07<00:17,  1.84it/s]

Evaluating:  79%|███████▉  | 122/154 [01:07<00:17,  1.83it/s]

Evaluating:  80%|███████▉  | 123/154 [01:08<00:17,  1.80it/s]

Evaluating:  81%|████████  | 124/154 [01:08<00:16,  1.82it/s]

Evaluating:  81%|████████  | 125/154 [01:09<00:15,  1.83it/s]

Evaluating:  82%|████████▏ | 126/154 [01:09<00:15,  1.79it/s]

Evaluating:  82%|████████▏ | 127/154 [01:10<00:14,  1.81it/s]

Evaluating:  83%|████████▎ | 128/154 [01:11<00:14,  1.81it/s]

Evaluating:  84%|████████▍ | 129/154 [01:11<00:13,  1.81it/s]

Evaluating:  84%|████████▍ | 130/154 [01:12<00:13,  1.82it/s]

Evaluating:  85%|████████▌ | 131/154 [01:12<00:12,  1.79it/s]

Evaluating:  86%|████████▌ | 132/154 [01:13<00:12,  1.79it/s]

Evaluating:  86%|████████▋ | 133/154 [01:13<00:11,  1.81it/s]

Evaluating:  87%|████████▋ | 134/154 [01:14<00:11,  1.81it/s]

Evaluating:  88%|████████▊ | 135/154 [01:14<00:10,  1.8

## Evaluate

Finally, we report the accuracy and F1-score metrics for each model, as well as the fine-tuning time in hours.

In [35]:
pd.DataFrame(results)

,distilbert-base-uncased,roberta-base,xlnet-base-cased
accuracy,0.901406,0.919536,0.925647
f1-score,0.897829,0.916793,0.923171
time,0.111936,0.189581,0.270957
